In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,analyzer='word', binary=True, min_df=10, max_df=.04)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.04, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'other', 'becomes', 'four', 'go', 'has', 'thick', 'often', 'system', 'by', 'cant', 'done', 'her', 'too', 'hereby', 'was', 'nevertheless', 'will', 'why', 'everything', 'seem', 'while', 'may', 'together', 'call', 'even', 'either', 'thru', 'in', 'every', 'between', 'fill', 'per', ...', 'yet', 'their', 'not', 'well', 'and', 'us', 'about', 'have', 'do', 'else', 'almost', 'we', 'eg'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [3]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 10299)

In [10]:
from tqdm import tqdm

def fun(X, number, a, b, n_iter=10):

    m1 = np.zeros((number, X.shape[1]))
    m2 = np.zeros((X.shape[0], number))
    m3 = np.zeros(number)
    print("*")
    docs, words = X.nonzero()
    z = np.random.choice(number, len(docs))
    
    for doc, word, z_ in zip(docs, words, z):
        m2[doc, z_] += 1
        m1[z_, word] += 1
        m3[z_] += 1
    
    for iter_ in tqdm(range(n_iter)):
        for i in range(len(docs)):
            m2[docs[i], z[i]] -= 1
            m1[z[i], words[i]] -= 1
            m3[z[i]] -= 1
            
            param = (m2[docs[i], :] + a) * (m1[:, words[i]] + b[words[i]]) / \
                (m3 + b.sum())
            z[i] = np.random.choice(np.arange(number), p=param / param.sum())
            
            m2[docs[i], z[i]] += 1
            m1[z[i], words[i]] += 1
            m3[z[i]] += 1
    
    return z, m1, m2, m3

In [11]:
number = 20
z, m1, m2, m3 = fun(X_train, number, 1 * np.ones(number), 1 * np.ones(X_train.shape[1]), 40)

*






  0%|          | 0/40 [00:00<?, ?it/s]



  2%|▎         | 1/40 [02:38<1:43:11, 158.75s/it]



  5%|▌         | 2/40 [05:38<1:44:27, 164.93s/it]



  8%|▊         | 3/40 [08:46<1:45:57, 171.82s/it]



 10%|█         | 4/40 [11:24<1:40:41, 167.81s/it]



 12%|█▎        | 5/40 [14:28<1:40:40, 172.58s/it]



 15%|█▌        | 6/40 [17:19<1:37:29, 172.06s/it]



 18%|█▊        | 7/40 [20:09<1:34:20, 171.52s/it]



 20%|██        | 8/40 [23:59<1:40:56, 189.26s/it]



 22%|██▎       | 9/40 [27:17<1:39:04, 191.77s/it]



 25%|██▌       | 10/40 [30:19<1:34:20, 188.70s/it]



 28%|██▊       | 11/40 [33:42<1:33:23, 193.22s/it]



 30%|███       | 12/40 [37:32<1:35:16, 204.16s/it]



 32%|███▎      | 13/40 [41:00<1:32:22, 205.29s/it]



 35%|███▌      | 14/40 [43:49<1:24:13, 194.37s/it]



 38%|███▊      | 15/40 [46:48<1:19:07, 189.89s/it]



 40%|████      | 16/40 [49:39<1:13:39, 184.16s/it]



 42%|████▎     | 17/40 [52:17<1:07:37, 176.41s/it]



 45%|████▌     | 18/40 [55:15<1:04:45, 176.62

In [13]:
top_words = np.argsort(m1, axis=1)[:, :-11:-1]

for tag in range(20):
    doc = np.zeros((1, X_train.shape[1]))
    for word in top_words[tag]:
        doc[0, word] = 1
    print(' Tag {}:\t{}'.format(tag+1, '\t'.join(vectorizer.inverse_transform(doc)[0])))

 Tag 1:	1993	april	clinton	house	including	national	president	states	today	white
 Tag 2:	chip	clipper	encryption	key	keys	phone	public	secret	secure	security
 Tag 3:	card	computer	disk	dos	hi	mac	memory	monitor	pc	video
 Tag 4:	answer	anybody	article	current	obvious	short	small	sorry	voltage	wondering
 Tag 5:	100	asking	condition	contact	offer	phone	price	sale	sell	shipping
 Tag 6:	children	country	history	israel	israeli	jews	killed	land	today	war
 Tag 7:	center	earth	launch	low	nasa	orbit	project	research	science	space
 Tag 8:	advance	anybody	guess	guys	haven	hey	level	ok	simple	sounds
 Tag 9:	application	code	file	files	ftp	graphics	server	user	version	window
 Tag 10:	banks	cause	doctor	effect	gordon	medical	pitt	soon	surrender	usually
 Tag 11:	cheers	comes	exactly	ideas	leave	longer	newsgroup	ok	sound	works
 Tag 12:	appreciated	btw	deleted	gets	hear	hey	oh	simply	stuff	thinking
 Tag 13:	11	12	13	14	17	18	21	23	24	25
 Tag 14:	bike	bought	car	cars	engine	fast	left	miles	road	turn
 Tag

In [14]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']